In [2]:
# An ideal prompting should follow the following format
# Instructions
# Contenxt
# Question
# Output

# Lets trye a prompt that follows the above format

prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library, also Google Cloud offers PaLM2 via api.These large language models have a large number of parameters. More the parameters more the complex the model is, but not necessarily efficient!

Question: Which libraries and model providers offer LLMs?

Answer: """

In [3]:
import os
import dotenv
dotenv.load_dotenv()

True

In [10]:
import openai 
api_key = os.getenv('OPENAI_API_KEY')
api_base = os.getenv('OPENAI_REVERSE_PROXY')

In [30]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=api_key,
             openai_api_base=api_base,
             )

In [31]:
print(llm(prompt))

 Hugging Face's `transformers` library, OpenAI using the `openai` library, Cohere using the `cohere` library, and Google Cloud offering PaLM2 via api.


In [36]:
# Since we dont know the query beforhand we will create a template for the futurre use

from langchain import PromptTemplate

prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library, also Google Cloud offers llms such as PaLM2 via api.These large language models have a large number of parameters. More the parameters more the complex the model is, but not necessarily efficient!

Question: {query}

Answer: """


template = PromptTemplate(input_variables=['query'],
                          template=prompt)

In [37]:
template.format(query="How many LLMs have been mentioned in the text")

'Answer the question based on the context below. If the\nquestion cannot be answered using the information provided answer\nwith "I don\'t know".\n\nContext: Large Language Models (LLMs) are the latest models used in NLP.\nTheir superior performance over smaller models has made them incredibly\nuseful for developers building NLP enabled applications. These models\ncan be accessed via Hugging Face\'s `transformers` library, via OpenAI\nusing the `openai` library, and via Cohere using the `cohere` library, also Google Cloud offers llms such as PaLM2 via api.These large language models have a large number of parameters. More the parameters more the complex the model is, but not necessarily efficient!\n\nQuestion: How many LLMs have been mentioned in the text\n\nAnswer: '

In [39]:
# template is ready lets feed it to the llml
llm(template.format(query="How many LLMs have been mentioned in the text?Name them"))

" Three LLMs have been mentioned in the text: Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library, as well as Google Cloud's PaLM2 via api."

In [41]:
# Parametric knowledge is the info the model has been trained with. source knowledge is the knowledge provided at the inference time via the prompt

# This is called Few shot learning

# First lets try this manually

prompt = """This is a conversation between a human an AI.The assistant is humorous and not really helpful in that sense.Here are some examples

User: What is the meaning of life?
AI: """

llm.temperature=1.0
llm(prompt=prompt)

' The meaning of life is a little bit different for everyone! However, I think the key is to find joy and satisfaction in whatever you choose to do.'

In [42]:
# With just one example, and being a humorous bot its answer is not so great, albeit serious

# So lets give it few more examples

prompt = """This is a conversation between a human an AI.The assistant is humorous and not really helpful in that sense.Here are some examples

User: Hi, How are you?
AI: Hola,Im fine, Como Estas.

User: What time of day is it?
AI: Its morning, Buesnos Dias.

User: What is the meaning of life?
AI: """

llm(prompt=prompt)   # That slightly better than before

" Hmm, the meaning of life. That's a deep one. Let me think...Yes, the meaning of life is to be happy and live your best life!"

In [52]:
# Now lets try few shot prompt template

from langchain import FewShotPromptTemplate

examples = [
    {
        'query': 'Hi, How are you?',
        'answer': 'Hola,Im fine, Como Estas, why are you? not is you?'
        },{
            'query': 'What time of day is it?',
            'answer': 'AI: Its morning, Buesnos Dias.Does morning comes first or night??'
            }
            ]


#Create an example template

example_template = """
User: {query}
AI: {answer}
"""


# Create a prompt example from above template

example_prompt = PromptTemplate(input_variables=['query','answer'],
                                template=example_template)

# we will break our main prompt into prefix and suffix

prefix = """This is a conversation between a human an AI.The assistant is humorous and not really helpful in that sense.Here are some examples:"""

suffix = """User:{query}
            AI: """

In [53]:
# Now create the few shot prompt template

few_shot_prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt=example_prompt,
    prefix  = prefix,
    suffix = suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [54]:
query = "What is the meaning of life?"
print(few_shot_prompt_template.format(query=query))

This is a conversation between a human an AI.The assistant is humorous and not really helpful in that sense.Here are some examples:


User: Hi, How are you?
AI: Hola,Im fine, Como Estas, why are you? not is you?



User: What time of day is it?
AI: AI: Its morning, Buesnos Dias.Does morning comes first or night??


User:What is the meaning of life?
            AI: 


In [55]:
print(llm(few_shot_prompt_template.format(query=query)))

 Life is a journey and the meaning of life is up to you to discover.But if you need some help,I am here for ya!


In [59]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What type of artificial intelligence do you use to handle complex tasks?",
        "answer": "I use a combination of cutting-edge neural networks, fuzzy logic, and a pinch of magic."
    }, {
        "query": "What is your favorite color?",
        "answer": "79"
    }, {
        "query": "What is your favorite food?",
        "answer": "Carbon based lifeforms"
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "What is the best thing in the world?",
        "answer": "The perfect pizza."
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "If you could do anything in the world what would you do?",
        "answer": "Take over the world, of course!"
    }, {
        "query": "Where should I travel?",
        "answer": "If you're looking for adventure, try the Outer Rim."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

In [60]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # this sets the max length that examples should be
)

In [67]:
import re

some_text = "There are a total of 8 words here.\n Plus three here.totalling eleven words"

words = re.split('[\n ]', some_text)

In [68]:
print(words, len(words))

['There', 'are', 'a', 'total', 'of', '8', 'words', 'here.', '', 'Plus', 'three', 'here.totalling', 'eleven', 'words'] 14


In [69]:
#Now create the few shot prompt template

dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=['query'],
    example_separator='\n')

In [70]:
print(dynamic_prompt_template.format(query="how do birds fly?"))

This is a conversation between a human an AI.The assistant is humorous and not really helpful in that sense.Here are some examples:

User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42

User:how do birds fly?
            AI: 


In [73]:
llm(dynamic_prompt_template.format(query="where can i find rapunzel?How is she now, since her hair is shorter now.How is she coming down from her tower? Does she use tech yet? What is her favourite electronic gadget?"))

"\nWell, that's a tall order. Rapunzel is usually found in the Happily Ever After realm. She's been doing great since her hair is shorter and her days of being locked in her tower are over. She still hasn't gotten modern technology but her favorite electronic gadget is definitely her trusty smartphone."